# How to save and restore a trained model

After scrolling through the posts of [reddit.com/r/learnmachinelearning](https://www.reddit.com/r/learnmachinelearning/), I've realized that the major bottlenecks of a machine learning project occur in the data input pipeline and in the final stage of the model, where you have to save the model and make predictions on new data.
So I thought that it would be useful to make a simple and straightforward tutorial to show you how you could save and restore a model that you have built with Tensorflow Eager.

### Tutorial flowchart
----

![img](tutorials_graphics/save_restore_model.png)

## Import here useful libraries

In [1]:
# Import TensorFlow and TensorFlow Eager
import tensorflow as tf
import tensorflow.contrib.eager as tfe

# Import function to generate toy classication problem
from sklearn.datasets import make_moons

In [2]:
# Enable eager mode. Once activated it cannot be reversed! Run just once.
tfe.enable_eager_execution()

## Part I: Build a simple neural network model for binary classification

In [3]:
class simple_nn(tf.keras.Model):
    def __init__(self):
        super(simple_nn, self).__init__()
        """ Define here the layers used during the forward-pass 
            of the neural network.
        """   
        # Hidden layer.
        self.dense_layer = tf.layers.Dense(10, activation=tf.nn.relu)
        # Output layer. No activation.
        self.output_layer = tf.layers.Dense(2, activation=None)
    
    def predict(self, input_data):
        """ Runs a forward-pass through the network.     
            Args:
                input_data: 2D tensor of shape (n_samples, n_features).   
            Returns:
                logits: unnormalized predictions.
        """
        hidden_activations = self.dense_layer(input_data)
        logits = self.output_layer(hidden_activations)
        return logits
    
    def loss_fn(self, input_data, target):
        """ Defines the loss function used during 
            training.         
        """
        logits = self.predict(input_data)
        loss = tf.losses.sparse_softmax_cross_entropy(labels=target, logits=logits)
        return loss
    
    def grads_fn(self, input_data, target):
        """ Dynamically computes the gradients of the loss value
            with respect to the parameters of the model, in each
            forward pass.
        """
        with tfe.GradientTape() as tape:
            loss = self.loss_fn(input_data, target)
        return tape.gradient(loss, self.variables)
    
    def fit(self, input_data, target, optimizer, num_epochs=500, verbose=50):
        """ Function to train the model, using the selected optimizer and
            for the desired number of epochs.
        """
        for i in range(num_epochs):
            grads = self.grads_fn(input_data, target)
            optimizer.apply_gradients(zip(grads, self.variables))
            if (i==0) | ((i+1)%verbose==0):
                print('Loss at epoch %d: %f' %(i+1, self.loss_fn(input_data, target).numpy()))

## Part II: Train model 

In [4]:
# Generate toy dataset for classification
# X is a matrix of n_samples x n_features and represents the input features
# y is a vector with length n_samples and represents our targets
X, y = make_moons(n_samples=100, noise=0.1, random_state=2018)
X_train, y_train = tf.constant(X[:80,:]), tf.constant(y[:80])
X_test, y_test = tf.constant(X[80:,:]), tf.constant(y[80:])

In [5]:
optimizer = tf.train.GradientDescentOptimizer(5e-1)
model = simple_nn()
model.fit(X_train, y_train, optimizer, num_epochs=500, verbose=50)

Loss at epoch 1: 0.663213
Loss at epoch 50: 0.260983
Loss at epoch 100: 0.248480
Loss at epoch 150: 0.245092
Loss at epoch 200: 0.240079
Loss at epoch 250: 0.227281
Loss at epoch 300: 0.196819
Loss at epoch 350: 0.153906
Loss at epoch 400: 0.113223
Loss at epoch 450: 0.083055
Loss at epoch 500: 0.061716


## Part III: Save trained model

In [6]:
# Specify checkpoint directory
checkpoint_directory = 'models_checkpoints/SimpleNN/'
# Create model checkpoint
checkpoint = tfe.Checkpoint(optimizer=optimizer,
                            model=model,
                            optimizer_step=tf.train.get_or_create_global_step())

In [7]:
# Save trained model
checkpoint.save(file_prefix=checkpoint_directory)

'models_checkpoints/SimpleNN/-1'

## Part IV: Restore trained model


In [8]:
# Reinitialize model instance
model = simple_nn()
optimizer = tf.train.GradientDescentOptimizer(5e-1)

In [9]:
# Specify checkpoint directory
checkpoint_directory = 'models_checkpoints/SimpleNN/'
# Create model checkpoint
checkpoint = tfe.Checkpoint(optimizer=optimizer,
                            model=model,
                            optimizer_step=tf.train.get_or_create_global_step())

In [10]:
# Restore model from latest chekpoint
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_directory))

## Part V: Check if the model was restored correctly

In [11]:
model.fit(X_train, y_train, optimizer, num_epochs=1)

Loss at epoch 1: 0.061338


The loss seems to be consistent with the loss we obtained in the last epoch of previous training :)!

## Part VI: Make predictions on new data

In [12]:
logits_test = model.predict(X_test)

In [13]:
print(logits_test)

tf.Tensor(
[[ 0.46661161 -0.81602877]
 [-2.69060157  1.23526029]
 [ 2.43001643 -2.35215769]
 [-2.86877127  0.42311112]
 [ 0.68118177 -0.80872488]
 [ 2.36590208 -2.16004146]
 [-2.80039488  2.21221565]
 [-3.16031675  0.71963893]
 [-2.76745803  0.67758663]
 [ 2.6786072  -2.55252435]
 [-1.11169555  0.52728788]
 [-3.76186803  1.23969049]
 [-2.98417084  0.51195994]
 [ 0.46995946 -0.60803168]
 [ 1.27858134 -1.52766214]
 [ 1.53762815 -1.62909484]
 [ 2.75608972 -2.49590808]
 [ 0.92804539 -1.04823756]
 [ 1.93477522 -2.09747617]
 [-2.98634282  0.35103911]], shape=(20, 2), dtype=float64)
